In [1]:
!pip install ipython-sql

You should consider upgrading via the 'c:\users\robert malka\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install pyodbc

You should consider upgrading via the 'c:\users\robert malka\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install mysql

You should consider upgrading via the 'c:\users\robert malka\anaconda3\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip install psycopg2

You should consider upgrading via the 'c:\users\robert malka\anaconda3\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip install sqlalchemy

You should consider upgrading via the 'c:\users\robert malka\anaconda3\python.exe -m pip install --upgrade pip' command.


In [6]:
import pyodbc
import psycopg2
import sqlalchemy
#import mysql

ModuleNotFoundError: No module named 'mysql'

In [7]:
!pip install sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
You should consider upgrading via the 'c:\users\robert malka\anaconda3\python.exe -m pip install --upgrade pip' command.


In [8]:
import sqlite3

In [51]:
# import sqlite3
from sqlite3 import Error
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
SELECT f.name, m.firstname||' '|| m.surname as Member,
COUNT(f.name) as bookings,
MONTH(starttime)
FROM Members m
INNER JOIN Bookings b on b.memid = m.memid
INNER JOIN Facilities f on f.facid = b.facid
WHERE m.memid > 0
GROUP BY f.name, m.firstname||' '|| m.surname
ORDER BY f.name, m.surname, m.firstname 

    """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks


OperationalError: no such function: MONTH

In [ ]:
-- All the queries I can copy into the above.

In [ ]:
/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */


/* Goal: pull up the member cost list, and then from that list, select the 
facilities that charge more than $0. Q1-5 uses the Facilities table.*/


SELECT name
FROM Facilities
WHERE membercost > 0

In [ ]:
/* Q2: How many facilities do not charge a fee to members? */


/*Goal: This is just the inverse of Q1: literally, pull all the facilities that 
charge $0, and then COUNT them. */

SELECT COUNT(name) #COUNT(*) will look across all columns for anything non-missing. Counting the rows that way.  
FROM Facilities
WHERE membercost = 0 

In [ ]:
/* Q3: Write a SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */


/*Goal: So we want to take the list from Q1, then divide the monthly maintenance
cost of those facilities from it. IF it's <20% of the maintenance cost, then we
show it in a new list. That list has to have the Facility ID, the name, the
member cost, and the monthly maintenance. */

SELECT facid, name, membercost, monthlymaintenance
FROM facilities
WHERE (membercost * 5) < monthlymaintenance

# membercost / monthlymaintenance < 0.2 is the other way to do it.

In [ ]:
/* Q4: Write a SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */


/*Goal: to start writing it with the OR operator. EITHER ID 1 OR ID 5.
Is there another way to do it? We'll talk to a mentor about that. */


SELECT *
FROM facilities
WHERE facid = 1 OR facid = 5  --how to do this without OR?

#WHERE facid IN 1, 5
#IN checks for every element in the list being provided.

In [ ]:

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

'''
SELECT name, monthlymaintenance
FROM facilities
CASE 
	WHEN monthlymaintenance < 100 THEN 'CHEAP' #i forget how to do this
	WHEN monthlymaintenance > 100 THEN 'EXPENSIVE'
	END AS label_for_maintenance_cost
'''

#The above will throw an error.

#We're creating a new column, which is why FROM is last, and END AS is needed, and there's a comma at the end of select (designating 3rd column)

SELECT name, monthlymaintenance,
CASE WHEN monthlymaintenance > 100
THEN 'expensive'
ELSE 'cheap'
END AS label_for_maintenance_cost
FROM Facilities


#END AS allows us to create a new variable and give it a name.


/*Goal: IF monthly maintenance cost of (ALL Facilities SELECTed) is >100,
THEN we add a column and label that "expensive;" ELSE we label it "cheap."
All facilities should show the name and monthly maintenance cost, in addition
to the label. */

In [ ]:
/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */


SELECT m.surname, m.firstname, m.joindate
FROM members AS m
ORDER BY joindate DESC
LIMIT 1 #or 3 if you want to make sure it is the most recent joindate &c

#The below is how to do it without LIMIT.

SELECT firstname, surname
FROM Members
WHERE joindate = (
	SELECT MAX(joindate)
	FROM Members )

/*Goal: (How do I do this one? Review the LIMIT clause & why it's
essential here.) You will now be using the MEMBERS table, and not 
the FACILITIES table. Use the Joindate column, and show the
firstname and surname from those columns. */

In [ ]:
/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */


SELECT m.firstname|| ','|| m.surname AS fullname, f.name
FROM members AS m 
JOIN Bookings AS b ON m.memid = b.memid
JOIN Facilities AS f ON b.facid = f.facid
WHERE b.facid IN (0, 1)
ORDER BY fullname DESC



#DISTINCT = no duplicates

/*Goal: From the MEMBERS table, -- I can't find the tennis court column or line.
I'll have to do some looking in order to find it. (is it in a different
table?) But the idea is the same as earlier question(s) -- look for the 
"TRUE" or "YES" answer in a column, cross-reference it with the firstname 
and surname columns, and join those columns so that you have one column
called fullname. ORDER BY member name. I found the tennis court 1 & 2,
it's in the name column of FACILITIES. So we have to JOIN that to the members
table and see which ones overlap (that's an INNER JOIN, right).*/

In [ ]:
/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */


#it runs, but nothing shows.

SELECT b.starttime, f.name, (m.firstname|| ','|| m.surname) AS fullname, cost
CASE
	WHEN m.memid = 0
	THEN b.slots * f.guestcost
    ELSE b.slots * f.membercost
	END AS cost
FROM bookings AS b 
JOIN facilities AS f ON b.facid = f.facid 
JOIN members AS m ON b.memid = m.memid
WHERE starttime >= '2012-09-14' 
    AND starttime < '2012-09-15'
    AND CASE
    WHEN m.memid = 0
	THEN b.slots * f.guestcost
    ELSE b.slots * f.membercost
	END > 30 
ORDER BY cost DESC    
   

-- First casewhen is to create the column
-- Second casewhen is to filter the information
   ''' 
    ((m.memid = 0
		AND b.slots * f.guestcost > 30)
	OR (m.memid != 0
		AND b.slots * f.membercost > 30))


#ON represents one or more JOIN conditions by which we could match records from one table to other.


#which will cost the member or guest more than $30.


/*Goal: Elaborate further: ORDER BY DESCENDING. Reference both members and 
guests. No subqueries.*/

In [40]:
/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT facility, fullname, cost
FROM (SELECT m.firstname|| ','|| m.surname AS fullname, f.name AS facility,
CASE
	WHEN m.memid = 0
	THEN b.slots * f.guestcost
    ELSE b.slots * f.membercost
	END AS cost
FROM members AS m 
JOIN facilities AS f ON b.facid = f.facid 
JOIN bookings AS b ON b.memid = m.memid
WHERE b.starttime >= '2012-09-14' 
    AND b.starttime < '2012-09-15')
    	WHERE cost > 30 
ORDER BY cost DESC

'''
SELECT member, facility, cost
FROM (SELECT Members.surname AS member, Facilities.name AS facility,
CASE
WHEN Members.memid =0
THEN Bookings.slots * Facilities.guestcost
ELSE Bookings.slots * Facilities.membercost
END AS cost
FROM Members
JOIN Bookings ON Members.memid = Bookings.memid
JOIN Facilities ON Bookings.facid = Facilities.facid
WHERE Bookings.starttime >= '2012-09-14'
AND Bookings.starttime < '2012-09-15') 
AS bookings
WHERE cost >30
ORDER BY cost DESC

    '''

#Review subqueries. Outer select statement, can replace that with cost >30


/*Goal: Q8 as a subquery.*/


# Note certain differences in SQLite, e.g. || instead of CONCAT. 

SyntaxError: invalid syntax (<ipython-input-40-2df34c4d9816>, line 1)

In [ ]:
/* PART 2: SQLite
/* We now want you to jump over to a local instance of the database on your machine. 

Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 

Make sure that the SQLFiles folder containing thes files is in your working directory, and
that you haven't changed the name of the .db file from 'sqlite\db\pythonsqlite'.

You should see the output from the initial query 'SELECT * FROM FACILITIES'.

Complete the remaining tasks in the Jupyter interface. If you struggle, feel free to go back
to the PHPMyAdmin interface as and when you need to. 

You'll need to paste your query into value of the 'query1' variable and run the code block again to get an output.
 
QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

SELECT f.name,
CASE
	WHEN m.memid = 0
	THEN b.slots * f.guestcost
	ELSE b.slots * f.membercost
	END AS revenue
FROM facilities as f 
JOIN bookings as b ON b.facid = f.facid 
JOIN members as m ON m.memid = b.memid
GROUP BY f.name -- before having because having is a condition (those come after group by)
HAVING revenue < 1000 
ORDER BY revenue


#WHERE tends to work for subqueries
-- do I need to join anything else?
-- is anything missing here?
-- in Q8/Q9, I was calling slots * membercost "cost." Same as revenue?


/* slots times member cost for that person (multiplying columns from two different tables to represent revenue.) */

In [ ]:
/* Q11: Produce a report of members and who recommended them in 
alphabetic surname,firstname order */

-- 


SELECT (m.surname|| ','|| m.firstname) AS recmd, (rcmd.firstname|| ','|| rcmd.surname) as Member
FROM members AS m
INNER JOIN Members rcmd on rcmd.recommendedby = m.memid

#ORDER BY member, recommended_by ASC


-- rcmd another name/alias for members, so match it with itself.
#Self-join is joining the table with itself.

In [ ]:
/* Q12: Find the facilities with their usage by member, but not 
guests */

SELECT f.name, (m.surname|| ','|| m.firstname) AS member, COUNT(*) 
FROM facilities AS f
JOIN bookings as b ON b.facid = f.facid 
JOIN members as m ON m.memid = b.memid
WHERE m.memid > 0
GROUP BY f.name, member
ORDER BY f.name -- usage the number of times it's being used, so no need to sub for revenue here.



'''
CASE WHEN b.memid != 0
	THEN COUNT(b.slots) -- need "distinct" here?
	END AS usage
FROM bookings as b

'''
-- is this finished?

/*Goal: I need to COUNT slots and assign them to their 
memid, so long as the memid isn't zero. */

In [ ]:
/* Q13: Find the facilities usage by month, but not guests */ -- but not guests?... 


#-- I don't even understand what this means since it's all in Sept?
# Legit don't know what they mean by usage. Could be the one booking, or the slots in that booking.


#Need: facility month sum(slots)

SELECT f.name, m.firstname||' '|| m.surname as Member,
COUNT(f.name) as bookings,
MONTH(starttime)
FROM members AS m
INNER JOIN Bookings b on b.memid = m.memid
INNER JOIN Facilities f on f.facid = b.facid
WHERE m.memid > 0
GROUP BY f.name, m.firstname||' '|| m.surname, MONTH(starttime)
ORDER BY f.name, m.surname, m.firstname 



#Do we need this? AND year(starttime) = 2012
#Subquery for tables we want to easily create and then pull from after.



'''
SELECT f.name, MONTH(f.starttime), SUM(b.slots)
WHERE m.memid > 0
FROM facilities as f
JOIN bookings as b ON b.facid = f.facid 
JOIN members as m ON m.memid = b.memid
GROUP BY f.name, m.surname|| ','|| m.firstname)

#ORDER BY f.name